# Bug triage with Deep Learning - PROPOSE

In [1]:
import keras

Using TensorFlow backend.


In [2]:
from __future__ import print_function, division

In [3]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
from annoy import AnnoyIndex
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [4]:
from keras.layers import Conv1D, Input, Add, Activation, Dropout, Embedding, MaxPooling1D, \
    GlobalMaxPool1D, Flatten, Dense, Concatenate, BatchNormalization
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras.layers.advanced_activations import LeakyReLU, ELU
from keras import optimizers

In [5]:
from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

## Auxiliary methods

## Configurações Globais

In [6]:
MAX_SEQUENCE_LENGTH_T = 20 # 40
MAX_SEQUENCE_LENGTH_D = 20 # 200
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000

'''
    Configuration
'''
epochs = 100
best_loss = 1
best_epoch = 0
verbose = 0
loss = 1

### Parse bugs preproprecessed

In [7]:
# Domain to use
DOMAIN = 'netbeans'
METHOD = 'propose_{}'.format(epochs)
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Path embeddings
EMBED_DIR='data/embed'
# Save model
SAVE_PATH = '{}_feature@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_feature_@number_of_epochs@epochs_64batch({})'.format(METHOD, DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [8]:
baseline = Baseline(DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

In [9]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

Creating the buckets...


#### Loading bug ids in memory

In [10]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


216715

#### Dicionário de títulos e descrições

In [11]:
%%time

experiment.load_bugs()
len(baseline.sentence_dict)


CPU times: user 16.8 s, sys: 2.18 s, total: 19 s
Wall time: 19 s


#### Hashing bugs by buckets

In [12]:
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare the train and test

In [13]:
experiment.prepare_dataset(issues_by_buckets, path_train='train_chronological', path_test='test_chronological')
# Read and create the test queries duplicates
retrieval.create_queries()

Reading train data
Reading bug ids


#### Recovery bug ids from train

In [14]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

#### Display a random bug

In [15]:
idx = np.random.choice(baseline.bug_ids, 1)[0]
baseline.bug_set[idx]

{'bug_severity': '0\n',
 'bug_status': '2\n',
 'component': '139\n',
 'creation_ts': '2007-11-14 14:14:00 +0000',
 'delta_ts': '2008-02-07 14:26:30 +0000',
 'description': '[CLS] product version : net ##be ##ans id ##e 6 . 0 rc ##1 ( build 2007 ##11 ##14 ##00 ##00 ) java : 1 . 6 . 0 _ 05 - ea ; java hot ##sp ##ot ( t ##m ) client v ##m 1 . 6 . 0 _ 05 - ea - b ##0 ##6 system : linux version 2 . 6 . 5 - 1 . 35 ##8 running on i ##38 ##6 ; ut ##f - 8 ; en _ us ( n ##b ) reference to outer class is replaced by null when access ##ing static members steps to reproduce : 1 ) have a class : public class new ##class { public static int field ; static class inner { public void m ( ) { field = 2 ; } } } 2 ) move inner to outer - > package p ; class inner { public void m ( ) { null . field = 2 ; / / < - - - - - - - - here should be new ##class . filed = 2 ; } } [SEP]',
 'description_bert': '[CLS] product version : net ##be ##ans id ##e 6 . 0 rc ##1 ( build 2007 ##11 ##14 ##00 ##00 ) java : 1 . 6 . 

### Generating the batch test

In [16]:
"Train ", len(baseline.dup_sets_train)

('Train ', 30600)

In [17]:
%%time

batch_size = 64
batch_size_test = 128

# we want a constant validation group to have a frame of reference for model performance
batch_triplets_valid, valid_input_sample, valid_input_pos, valid_input_neg, valid_sim = experiment.batch_iterator(None,
                                                                                          baseline.train_data, 
                                                                                          baseline.dup_sets_train,
                                                                                          bug_train_ids,
                                                                                          batch_size_test, 1,
                                                                                          issues_by_buckets)
test_gen = ([valid_input_sample['title'], valid_input_pos['title'], valid_input_neg['title'], 
             valid_input_sample['description'], valid_input_pos['description'], valid_input_neg['description'],
            valid_input_sample['info'], valid_input_pos['info'], valid_input_neg['info']], valid_sim)

# Categorical columns
number_of_columns_info = valid_input_sample['info'].shape[1]
# Max sequence title
MAX_SEQUENCE_LENGTH_T = valid_input_sample['title'].shape[1]
MAX_SEQUENCE_LENGTH_D = valid_input_sample['description'].shape[1]

CPU times: user 52.8 ms, sys: 16 µs, total: 52.8 ms
Wall time: 52.5 ms


In [18]:
valid_input_sample['title'].shape, valid_input_sample['description'].shape, valid_input_sample['info'].shape, valid_sim.shape

((128, 20), (128, 20), (128, 544), (128,))

### Validar entrada

In [19]:
# %%time 

# baseline.display_batch(baseline.train_data, baseline.dup_sets_train, bug_train_ids, 5)

## Pre-trained embeddings

Loading pretrained word vectors

### Fasttext

In [20]:
vocab = baseline.load_vocabulary(os.path.join(DIR, 'vocab_embed.pkl'))
#print(np.random.choice(vocab, 10))
# for token in vocab:
#     print(token)

vocabulary loaded


In [21]:
"Total vocabulary: {}".format(len(vocab))

'Total vocabulary: 19061'

In [22]:
def generating_embed(baseline, EMBED_DIR, EMBEDDING_DIM):
    embeddings_index = {}
    embed_path = os.path.join(EMBED_DIR, 'glove.42B.300d.txt')
    f = open(embed_path, 'rb')
    #num_lines = sum(1 for line in open(embed_path, 'rb'))

    vocab = baseline.load_vocabulary(os.path.join(baseline.DIR, 'vocab_embed.pkl'))
    vocab_size = len(vocab) 

    # Initialize uniform the vector considering the Tanh activation
    embedding_matrix = np.random.uniform(-1.0, 1.0, (vocab_size, EMBEDDING_DIM))
    embedding_matrix[0, :] = np.zeros(EMBEDDING_DIM)

    loop = tqdm(f)
    loop.set_description("Loading Glove")
    for line in loop:
        tokens = line.split()
        word = tokens[0]
        embeddings_index[word] = np.asarray(tokens[1:], dtype='float32')
        loop.update(1)
    f.close()
    loop.close()

    print('Total %s word vectors in Glove 42B 300d.' % len(embeddings_index))

    loop = tqdm(total=vocab_size)
    loop.set_description('Loading embedding from dataset pretrained')
    i = 0
    for word, embed in vocab.items():
        if word in embeddings_index:
            embedding_matrix[i] = embeddings_index[word]
        else:
            embedding_matrix[i] = np.asarray(embed, dtype='float32')
        loop.update(1)
        i+=1
    loop.close()
    baseline.embedding_matrix = embedding_matrix

In [23]:
%%time

generating_embed(baseline, EMBED_DIR=EMBED_DIR, EMBEDDING_DIM=EMBEDDING_DIM) # MAX_NB_WORDS=MAX_NB_WORDS

vocabulary loaded



Total 1917494 word vectors in Glove 42B 300d.



CPU times: user 1min 21s, sys: 2.66 s, total: 1min 23s
Wall time: 1min 23s


## Experiment

## Propose

https://github.com/tqtg/DuplicateBugFinder

In [24]:
from keras.initializers import RandomUniform, RandomNormal, Ones

### Embedding layer

In [25]:
from keras.constraints import MaxNorm
from keras.initializers import TruncatedNormal, RandomUniform

# Is missing the padding_idx used in pytorch
# https://pytorch.org/docs/stable/_modules/torch/nn/modules/sparse.html
# https://stackoverflow.com/questions/54824768/rnn-model-gru-of-word2vec-to-regression-not-learning
def embedding_layer(embeddings, num_words, embedding_dim, max_sequence_length, trainable, name):
    embedding_layer = Embedding(num_words,
                                  embedding_dim,
                                  name='embedding_layer_{}'.format(name),
                                  weights=[embeddings],
                                  embeddings_constraint=MaxNorm(max_value=1, axis=0),
                                  #input_length=max_sequence_length,
                                  input_length=None,
                                  trainable=trainable)
    return embedding_layer

### CNN Dilated

In [26]:
from keras.constraints import max_norm
import math

def DC_CNN_Block(nb_filter, filter_length, dilation, l2_layer_reg):
    def block(block_input):        
        residual =    block_input
        
        layer_out =   Conv1D(filters=nb_filter, kernel_size=filter_length, 
                      dilation_rate=dilation, 
                      activation='linear', padding='causal', use_bias=False)(block_input) #kernel_regularizer=l2(l2_layer_reg)                    
        
        activation_out = Activation('tanh')(layer_out)
        
        skip_out =    Conv1D(1,1, activation='linear', use_bias=False)(activation_out) # use_bias=False, kernel_constraint=max_norm(1.)
        
        c1x1_out =    Conv1D(1,1, activation='linear', use_bias=False)(activation_out)
                      
        block_out =   Add()([residual, c1x1_out])
        
        return block_out, skip_out
    return block

def cnn_dilated_model(embedding_layer, title_layer, max_sequence_length):
    sequence_input = Input(shape=(max_sequence_length, ), name='Feature_BugInput_CNND')
    embedded_sequences = embedding_layer(sequence_input)

    units = 128
    number_of_layers = 6
    
    title_input = title_layer.input
    title_layer = title_layer.output

    # Embedding layer with CNN dilated
    #la, lb = DC_CNN_Block(units,2,1,0.01)(embedded_sequences)
    la = embedded_sequences
    la_title = title_layer
    attention_layes, attention_title_layes = [], []
    filters_size = [3, 4, 5]
    number_of_filters = len(filters_size)
    for index in range(1, number_of_layers + 1):
        # Desc
        la, lb = DC_CNN_Block(units, 5, int(math.pow(2, index)), 0.01)(la)
        # Title 
        la_title, lb_title = DC_CNN_Block(units, 3, int(math.pow(2, index)), 0.01)(la_title)
        lb = Add()([lb_title, lb])
        #la = Dropout(.90)(la)
        #lb = Dropout(.90)(lb)
        attention_layes.append(lb)
        attention_title_layes.append(lb_title)

    attention_layer = Add()(attention_layes)
    attention_title_layes = Add()(attention_title_layes)
    attention_layer =   Add()([attention_layer, attention_title_layes])
    
    #layer = Add()([attention_layer, l9])
    
    layer =   Activation('tanh')(attention_layer)

    #layer =  Conv1D(1,1, activation='linear', use_bias=False)(layer)
    
    #layer = Flatten()(layer)
    layer = GlobalAveragePooling1D()(layer)
    #layer = Dropout(0.50)(layer)
    layer = Dense(300, activation='tanh')(layer)
    #layer = GRU(150, activation='tanh', return_sequences=False)(layer)

    cnn_dilated_feature_model = Model(inputs=[sequence_input, title_input], outputs=[layer], name = 'FeatureCNNDilatedGenerationModel') # inputs=visible
    return cnn_dilated_feature_model

### CNN with filter 3,4,5

In [27]:
import keras
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, AveragePooling1D

def residual_context():
    def block(block_input):
        residual = block_input
        
        layer = block_input
        for filter_size in [32, 64]:
            #conv = Conv1D(filters=32, kernel_size=filter_size, activation='tanh')(conv)
            layer = Dense(filter_size, activation='tanh')(layer)
        shape_size = K.int_shape(block_input)[1]
        skip_out = Dense(shape_size, activation='linear', use_bias=False)(layer)
        block_out = Dense(shape_size, activation='linear', use_bias=False)(layer)
        
        block_out = Add()([residual, block_out])
        return block_out, skip_out
    return block

def cnn_model(embedding_layer, title_input, title_layer, max_sequence_length):

    sequence_input = Input(shape=(max_sequence_length,), name='Feature_BugInput_CNN')
    #sequence_input = Input(shape=(None,), name='Feature_BugInput')
    embedded_sequences = embedding_layer(sequence_input)

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    # best combination filter (3, 4, 5) e 128 e 256
    convs = []
    filter_sizes = [3, 4, 5]
    n_filters = 128

    for index, filter_size in enumerate(filter_sizes):
        l_conv = Conv1D(filters=n_filters, kernel_size=filter_size)(embedded_sequences)
        l_pool = MaxPooling1D(pool_size=filter_size)(l_conv) # index+1
        convs.append(l_pool)

    l_merge = Concatenate(axis=1)(convs)

    # add a 1D convnet with global maxpooling, instead of Yoon Kim model
    #conv = Conv1D(filters=32, kernel_size=5)(l_merge)
    if title_layer == None:
        #title_layer = Permute((2, 1))(title_layer)
        #conv = Permute((2, 1))(conv)
        #layer = Dot(axes=1)([conv, title_layer])
        #title_layer = TimeDistributed(Dense(1))(title_layer)
        title_layer = GlobalAveragePooling1D()(title_layer)
        layer = GlobalAveragePooling1D()(l_merge)
        layer = Concatenate()([layer, title_layer])
        #layer = Dropout(0.50)(layer)
        #layer = Activation('tanh')(layer)
        layer, layer_b  = residual_context()(layer)
        layer = Dropout(0.50)(layer)
        layer, layer_c  = residual_context()(layer)
        layer = Dropout(0.50)(layer)
        layer = Add()([layer_b, layer_c])
        layer = Activation('tanh')(layer)
    else:
        layer = GlobalAveragePooling1D()(l_merge)
    #layer = GlobalAveragePooling1D()(layer)
    #layer = Flatten()(l_merge)
    #layer = Activation('tanh')(layer)
    layer = Dense(300, activation='tanh')(layer)
    #layer = GRU(100, activation='tanh', return_sequences=False)(l_merge)
    #layer = LeakyReLU()(layer)
    
    if title_layer == None:
        inputs = [sequence_input, title_input]
    else:
        inputs = [sequence_input]

    cnn_feature_model = Model(inputs=inputs, outputs=[layer], name = 'FeatureCNNGenerationModel') # inputs=visible

    return cnn_feature_model

In [28]:
# desc_feature_model.summary()

### Bi-LSTM

In [29]:
from keras.layers import Dense, Input, LSTM, GRU, Dropout, Bidirectional, GlobalAveragePooling1D, Permute, Dot

def bilstm_model(embedding_layer, max_sequence_length):
    number_lstm_units = 50
    rate_drop_lstm = 0
    recurrent_dropout = 0

    sequence_input = Input(shape=(max_sequence_length, ), name='Feature_BugInput')
    #sequence_input = Input(shape=(None, ), name='Feature_BugInput')
    embedded_sequences = embedding_layer(sequence_input)

    # Creating LSTM Encoder
#     lstm_layer = Bidirectional(LSTM(number_lstm_units, return_sequences=True), # dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm 
#                                merge_mode='ave')

    left_layer = LSTM(number_lstm_units, return_sequences=True)(embedded_sequences)
    right_layer = LSTM(number_lstm_units, return_sequences=True, go_backwards=True)(left_layer)
    
    lstm_layer = Add()([left_layer, right_layer])
    
    #lstm_layer = TimeDistributed(Dense(1))(lstm_layer)
    #layer = Flatten()(lstm_layer)
    layer = GlobalAveragePooling1D()(lstm_layer)
    layer = Dense(300, activation='tanh')(layer)

    lstm_feature_model = Model(inputs=[sequence_input], outputs=[layer], name = 'FeatureLstmGenerationModel') # inputs=visible

    return lstm_feature_model, lstm_layer

In [30]:
# title_feature_model.summary()

### MLP

In [31]:
def mlp_model(input_size):
    info_input = Input(shape=(input_size, ), name='Feature_BugInput')
    input_size = 300
    
    for units in [64, 32]:
        layer = Dense(units, activation='tanh', kernel_initializer='random_uniform')(info_input)
    
    layer = Dense(input_size, activation='tanh')(info_input)
    
    mlp_feature_model = Model(inputs=[info_input], outputs=[layer], name = 'FeatureMlpGenerationModel')
    
    return mlp_feature_model

### Siamese model

In [32]:
from keras import backend as K
import tensorflow as tf

'''
    Some loss ideas
    hinge loss Kullback-Leibler
    https://stackoverflow.com/questions/53581298/custom-combined-hinge-kb-divergence-loss-function-in-siamese-net-fails-to-genera
'''

def normalize(x, axis):
    norm = K.sqrt(K.sum(K.square(x), axis=axis, keepdims=False))
    return x, K.maximum(norm, K.epsilon())
    
# https://github.com/keras-team/keras/issues/3031
# https://github.com/keras-team/keras/issues/8335
def cosine_distance(inputs):
    x, y = inputs
    x, x_norm = normalize(x, axis=-1)
    y, y_norm = normalize(y, axis=-1)
    distance = K.sum( x * y, axis=-1) / (x_norm * y_norm)
    distance = (distance + K.constant(1)) / K.constant(2)
    # Distance goes from 0 to 2 in theory, but from 0 to 1 if x and y are both
    # positive (which is the case after ReLU activation).
    return K.mean(distance, axis=-1, keepdims=False)

def euclidean_distance(vects):
    x, y = vects
    distance = K.sqrt(K.maximum(K.sum(K.square(x - y), axis=1, keepdims=True), K.epsilon()))
    # Normalize https://stats.stackexchange.com/questions/53068/euclidean-distance-score-and-similarity
    distance = K.constant(1) / (K.constant(1) + distance)
    return K.mean(distance, keepdims=False)
    #return K.mean(distance, axis=-1, keepdims=False)

# https://jdhao.github.io/2017/03/13/some_loss_and_explanations/
def triplet_loss(y_true, y_pred):
    margin = K.constant(1.0)
    pos = y_pred[0]
    neg = y_pred[1]
    return K.mean(K.maximum(0.0, pos - neg + margin))

def custom_margin_loss(y_true, y_pred):
    margin = K.constant(1.0)
    pos = y_pred[0]
    neg = y_pred[1]
    return K.mean(K.maximum(0.0, margin - pos + neg), keepdims=False)

# https://www.kaggle.com/c/quora-question-pairs/discussion/33631
# https://www.researchgate.net/figure/Illustration-of-triplet-loss-contrastive-loss-for-negative-samples-and-binomial_fig2_322060548
def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    pos = y_pred[0]
    neg = y_pred[1]
    margin = 1
    return K.mean(pos * K.square(neg) +
                  (1 - pos) * K.square(K.maximum(margin - neg, 0)))

def pos_distance(y_true, y_pred):
    return y_pred[0]

def neg_distance(y_true, y_pred):
    return y_pred[1]

def stack_tensors(vects):
    return K.stack(vects, axis=-1)

#### Propose

In [33]:
from keras.initializers import TruncatedNormal
from keras.regularizers import l2
from keras.layers import Average, Dot, Maximum, Permute, Reshape

def residual_bug():
    def block(block_input):
        shape_size_cols = K.int_shape(block_input)[1]
        shape_size_rows = 1
        
        residual =  block_input
        residual = Activation('relu')(residual)
        #residual = BatchNormalization()(residual)
        
        layer_out = Reshape((shape_size_cols, shape_size_rows))(block_input)
        layer_out = GRU(100, activation='tanh', return_sequences=True)(layer_out)
        #right_layer_out = GRU(100, activation='tanh', return_sequences=True, go_backwards=True)(left_layer_out)
        #layer_out = Add()([left_layer_out, right_layer_out])
        #layer_out = Reshape((shape_size_cols, ))(layer_out)
        layer_out = GlobalAveragePooling1D()(layer_out)
        #layer_out = BatchNormalization()(layer_out)
        layer_out = Dense(50, activation='tanh')(layer_out)
        #layer_out = BatchNormalization()(layer_out)
        layer_out = Dense(shape_size_cols, activation='tanh', use_bias=True)(layer_out)
        skip_out = Dense(shape_size_cols, activation='tanh', use_bias=True)(layer_out)
        #layer_out = Activation('relu')(layer_out)
        #layer_out = BatchNormalization()(layer_out)
        
        block_out = Add()([residual, layer_out])
        #block_out = Activation('relu')(block_out)
        return block_out, skip_out
    return block

In [34]:
from keras.layers import concatenate, Add, Lambda, merge, Average, Maximum, Subtract, Average
from keras.optimizers import Adam, Nadam

def siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, sequence_length_info, 
                  sequence_length_t, sequence_length_d, name):
  
    bug_t = Input(shape = (sequence_length_t, ), name = 'title_{}'.format(name))
    bug_d = Input(shape = (sequence_length_d, ), name = 'desc_{}'.format(name))
    bug_i = Input(shape = (sequence_length_info, ), name = 'info_{}'.format(name))
    
    bug_t_feat = title_feature_model(bug_t)
    bug_d_feat = desc_feature_model(bug_d)
    #bug_d_feat = desc_feature_model([bug_d, bug_t])
    bug_i_feat = categorical_feature_model(bug_i)
    
    #bug_t_feat = GlobalAveragePooling1D()(bug_t_feat)
    
#     encoded_t_1a, encoded_t_1b  = residual_bug()(bug_t_feat)
#     encoded_d_1a, encoded_d_1b  = residual_bug()(bug_d_feat)
#     bug_t_feat = encoded_t_1a
#     bug_d_feat = encoded_d_1a
    
    #bug_feature_output = Add(name = 'merge_features_{}'.format(name))([bug_i_feat, bug_t_feat, bug_d_feat])
    bug_feature_output = concatenate([bug_i_feat, bug_t_feat, bug_d_feat], name = 'merge_features_{}'.format(name))
    
    bug_feature_output, bug_feature_output_1b = residual_bug()(bug_feature_output)
    #bug_feature_output_1a = Dropout(.5)(bug_feature_output_1a)
    #bug_feature_output, bug_feature_output_2b = residual_bug()(bug_feature_output_1a)
    
    #bug_feature_output = Add()([bug_feature_output_1b, bug_feature_output_2b])
    #bug_feature_output = BatchNormalization()(bug_feature_output)
    #bug_feature_output = Activation('relu')(bug_feature_output)
#     bug_feature_output = Dropout(.75)(bug_feature_output)
#     shape_size = K.int_shape(bug_feature_output)[1]
#     bug_feature_output = Dense(shape_size, activation='linear', use_bias=False)(bug_feature_output)
#     bug_feature_output = Dropout(.33)(bug_feature_output)
#     bug_feature_output = Dense(100)(bug_feature_output)
    
    #bug_feature_output  = residual_bug()(bug_feature_output)
    #bug_feature_output = BatchNormalization()(bug_feature_output)
    #     encoded_2a, encoded_2b  = residual_bug()(encoded_1a)
    
    #     bug_feature_output = Add()([encoded_1b, encoded_2b])
    #     bug_feature_output = Activation('tanh')(bug_feature_output)
    
    # Bug representation layer
    # bug_feature_output = Dense(300, activation='tanh')(bug_feature_output)
    
    bug_feature_model = Model(inputs=[bug_t, bug_d, bug_i], outputs=[bug_feature_output], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

In [35]:
def max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=1):
    
    inputs = np.concatenate([encoded_anchor.input, encoded_positive.input, encoded_negative.input], -1).tolist()
    
    encoded_anchor = encoded_anchor.output
    encoded_positive = encoded_positive.output
    encoded_negative = encoded_negative.output
    
    # Distance
    positive_d = Lambda(cosine_distance, name='pos_cosine_distance', output_shape=[1])([encoded_anchor, encoded_positive])
    negative_d = Lambda(cosine_distance, name='neg_cosine_distance', output_shape=[1])([encoded_anchor, encoded_negative])

    # Loss function only works with a single output
    output = Lambda(
        lambda vects: stack_tensors(vects),
        name='stack-distances',
        output_shape=(2, 1)
    )([positive_d, negative_d])
    
    #loss = MarginLoss()(output)

    similarity_model = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

    #optimizer = Nadam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=K.epsilon(), schedule_decay=0.01)
    optimizer = Adam(lr=1e-3 * decay_lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

    # setup the optimization process 
    similarity_model.compile(optimizer='adam', loss=custom_margin_loss, metrics=[pos_distance, neg_distance, custom_margin_loss])

    return similarity_model

In [36]:
%%time
import keras

# Inspired on https://'pastebin.com/TaGFdcBA
# TODO: https://stackoverflow.com/questions/49941903/keras-compute-cosine-distance-between-two-flattened-outputs
keras.backend.clear_session()

# Clear GPU memory
# from numba import cuda
# cuda.select_device(0)
# cuda.close()

# Embeddings
desc_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(baseline.embedding_matrix), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_D, 
                              trainable=False, name='desc')
title_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(baseline.embedding_matrix), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_T, 
                              trainable=False, name='title')

# Feature models
'''
    cnn_dilated_model
    arcii_model
    cnn_model
    lstm_model
    bilstm_model
'''
title_feature_model, title_layer = bilstm_model(title_embedding_layer, MAX_SEQUENCE_LENGTH_T)
desc_feature_model = cnn_model(desc_embedding_layer, title_feature_model.input, 
                               title_layer, MAX_SEQUENCE_LENGTH_D)
categorical_feature_model = mlp_model(number_of_columns_info)

# Similarity model
encoded_anchor = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'in')
encoded_positive = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'pos')
encoded_negative = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'neg')

similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=1)

# cnn_feature_model.summary()
# lstm_feature_model.summary()
similarity_model.summary()

'''
    Experiment
'''
for epoch in range(epochs):
    batch_triplet_train, \
        train_input_sample, train_input_pos, train_input_neg, \
            train_sim = experiment.batch_iterator(encoded_anchor, baseline.train_data, baseline.dup_sets_train, 
                                                  bug_train_ids, batch_size, 1, issues_by_buckets)
    train_batch = [train_input_sample['title'], train_input_sample['description'], train_input_sample['info'],
                   train_input_pos['title'], train_input_pos['description'], train_input_pos['info'], 
                   train_input_neg['title'], train_input_neg['description'], train_input_neg['info']]
    
#     if epoch == 10:
#         similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=0.1)
    
    h = similarity_model.train_on_batch(x=train_batch, y=train_sim)
    
    if (epoch+1 == epochs): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, _, debug = experiment.evaluate_validation_test(retrieval, verbose, encoded_anchor, issues_by_buckets, bug_train_ids)
        print("Epoch: {} Loss: {:.2f}, CustomLoss: {:.2f}, pos_cosine: {:.2f}, neg_cosine: {:.2f}, recall@25: {:.2f}".format(epoch+1,
                                                                                                         h[0],  h[3],
                                                                                                         h[1], h[2], recall))
    else:
        print("Epoch: {} Loss: {:.2f}, CustomLoss: {:.2f}, pos_cosine: {:.2f}, neg_cosine: {:.2f}".format(epoch+1,
                                                                                                         h[0],  h[3],
                                                                                                         h[1],
                                                                                                         h[2]))
    loss = h[3]
    
    if loss < best_loss:
        best_loss = loss
        best_epoch = epoch+1

experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
print('Best_epoch={}, Best_loss={:.2f}, Recall@25={:.2f}'.format(best_epoch, best_loss, recall))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 544)          0                                            
__________________________________________________________________________________________________
title_in (InputLayer)           (None, 20)           0                                            
__________________________________________________________________________________________________
desc_in (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
info_pos (InputLayer)           (None, 544)          0                                            
__________________________________________________________________________________________________
title_pos 

Epoch: 1 Loss: 1.03, CustomLoss: 1.03, pos_cosine: 0.92, neg_cosine: 0.95
Epoch: 2 Loss: 1.01, CustomLoss: 1.01, pos_cosine: 0.95, neg_cosine: 0.96
Epoch: 3 Loss: 1.01, CustomLoss: 1.01, pos_cosine: 0.95, neg_cosine: 0.96
Epoch: 4 Loss: 1.02, CustomLoss: 1.02, pos_cosine: 0.96, neg_cosine: 0.97
Epoch: 5 Loss: 1.00, CustomLoss: 1.00, pos_cosine: 0.97, neg_cosine: 0.97
Epoch: 6 Loss: 1.01, CustomLoss: 1.01, pos_cosine: 0.97, neg_cosine: 0.98
Epoch: 7 Loss: 1.00, CustomLoss: 1.00, pos_cosine: 0.98, neg_cosine: 0.98
Epoch: 8 Loss: 1.00, CustomLoss: 1.00, pos_cosine: 0.98, neg_cosine: 0.98
Epoch: 9 Loss: 1.00, CustomLoss: 1.00, pos_cosine: 0.98, neg_cosine: 0.98
Epoch: 10 Loss: 0.99, CustomLoss: 0.99, pos_cosine: 0.98, neg_cosine: 0.97
Epoch: 11 Loss: 0.99, CustomLoss: 0.99, pos_cosine: 0.98, neg_cosine: 0.97
Epoch: 12 Loss: 0.98, CustomLoss: 0.98, pos_cosine: 0.98, neg_cosine: 0.96
Epoch: 13 Loss: 0.97, CustomLoss: 0.97, pos_cosine: 0.98, neg_cosine: 0.95
Epoch: 14 Loss: 0.95, CustomLoss: 

In [37]:
_[:20]

['233472:230872|234329:0.9855268690735102,231856:0.9848450804129243,230092:0.9845450390130281,230090:0.9842839352786541,234917:0.9837376661598682,234918:0.9837376661598682,231804:0.9831626061350107,229428:0.9829104468226433,235223:0.9752777554094791,235224:0.9749243054538965,238521:0.9747538566589355,233848:0.9745042137801647,231419:0.9744695760309696,233852:0.9743472021073103,232979:0.9740253929048777,235782:0.973811898380518,234525:0.9732354823499918,230929:0.9731643628329039,234218:0.972349502146244,233727:0.9722915794700384,233709:0.9721514564007521,235129:0.9718491621315479,236741:0.9717494733631611,232615:0.9716611187905073,236716:0.9715937785804272,227180:0.9714547265321016,239024:0.9714519809931517,232621:0.9714026991277933,234631:0.9709947388619184',
 '230872:233472|235293:0.9826061353087425,229581:0.9811638556420803,229528:0.9724440947175026,235463:0.9674784354865551,234525:0.9666150026023388,235128:0.9655389040708542,234962:0.9645094200968742,237171:0.9644129388034344,230152

In [38]:
#recall, _, debug = experiment.evaluate_validation_test(retrieval, verbose, encoded_anchor, issues_by_buckets, bug_train_ids)
recall

0.7

In [39]:
experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)

Saved model 'modelos/model_propose_100_feature_100epochs_64batch(netbeans).h5' to disk


### Retrieval evaluation

In [40]:
print("Total of queries:", len(retrieval.test))

Total of queries: 3162


#### Getting the model trained

In [41]:
SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs))

'propose_100_feature_100epochs_64batch(netbeans)'

In [42]:
model = experiment.get_model_vectorizer(path=SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)))

/usr/local/lib/python3.5/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [43]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 544)          0                                            
__________________________________________________________________________________________________
title_in (InputLayer)           (None, 20)           0                                            
__________________________________________________________________________________________________
desc_in (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
FeatureMlpGenerationModel (Mode (None, 300)          163500      info_in[0][0]                    
__________________________________________________________________________________________________
FeatureLst

In [44]:
recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, 0, model, issues_by_buckets, bug_train_ids)

In [45]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

'data/processed/netbeans/exported_rank_propose_100.txt'

In [46]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [47]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report

{'1 - recall_at_5': 0.58,
 '2 - recall_at_10': 0.63,
 '3 - recall_at_15': 0.66,
 '4 - recall_at_20': 0.69,
 '5 - recall_at_25': 0.7}